In [3]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [4]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [5]:
subject = 'Piauí - Consumo de Cimento (t)'
split_index = 17 #Referente aos 18 anos de input  

In [6]:
data = pd.read_csv('2003_01_model_input_PI.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Ano,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),Atmospheric Pressure(mB),...,Air Relative Humidity (%),Wind Gust (m/s),Piauí - Produção de Cimento (t),Piauí - Desemprego,Piauí - PIB - Estadual,Piauí - PIB - Construção Civil,Piauí - PIB - Per Capita,Piauí - PIB - Preços de Mercado,Piauí - IDH,Piauí - Consumo de Cimento (t)
0,2003,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,0.895303,1.662891,1000.394769,...,75.511112,1.137362,317.286319,8.192266,2.136063e+07,1.358044e+06,6.061141,1.868149e+07,0.649218,238.886000
1,2004,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,0.746570,0.164626,1001.383287,...,76.766402,1.216410,351.667485,8.118950,2.172536e+07,1.379534e+06,6.108224,1.884442e+07,0.654538,276.711000
2,2005,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,0.589811,0.807185,1002.804857,...,62.893680,1.622106,357.819089,8.098198,2.207657e+07,1.395739e+06,6.154515,1.899957e+07,0.659288,272.764000
3,2006,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,0.606044,8.779783,1002.667748,...,59.586205,1.610569,386.232189,8.006233,2.240750e+07,1.409503e+06,6.196868,1.913991e+07,0.663482,333.178000
4,2007,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,0.633401,18.249461,1002.210356,...,62.656563,1.453108,445.515113,7.980382,2.268410e+07,1.390317e+06,6.221036,1.922183e+07,0.667129,386.585000
5,2008,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,0.587619,24.309801,1002.245680,...,67.391458,1.313538,510.644488,7.951466,2.296156e+07,1.400359e+06,6.253673,1.932330e+07,0.670241,449.076000
6,2009,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,0.566759,23.280755,1002.158424,...,69.383573,1.242112,564.654167,7.981121,2.318896e+07,1.418401e+06,6.277339,1.939333e+07,0.672829,536.766000
7,2010,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,0.593537,19.186561,1002.280740,...,70.066176,1.226949,595.360558,7.946167,2.385426e+07,1.441334e+06,6.336644,1.960635e+07,0.674904,639.535000
8,2011,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,0.600821,18.109353,1002.394173,...,69.081412,1.247464,612.063590,7.934312,2.447423e+07,1.448568e+06,6.385667,1.978514e+07,0.676479,625.022000
9,2012,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,0.604731,18.253900,1002.463449,...,67.999911,1.231408,616.463496,9.098665,2.502337e+07,1.450923e+06,6.419744,1.991803e+07,0.677105,687.536000


In [7]:
input_data = data.iloc[:split_index + 1, 1:-1]
input_data

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),...,Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Piauí - Produção de Cimento (t),Piauí - Desemprego,Piauí - PIB - Estadual,Piauí - PIB - Construção Civil,Piauí - PIB - Per Capita,Piauí - PIB - Preços de Mercado,Piauí - IDH
0,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,0.895303,1.662891,1000.394769,174.703219,...,26.657973,75.511112,1.137362,317.286319,8.192266,2.136063e+07,1.358044e+06,6.061141,1.868149e+07,0.649218
1,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,0.746570,0.164626,1001.383287,242.605103,...,25.082951,76.766402,1.216410,351.667485,8.118950,2.172536e+07,1.379534e+06,6.108224,1.884442e+07,0.654538
2,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,0.589811,0.807185,1002.804857,471.414128,...,21.817835,62.893680,1.622106,357.819089,8.098198,2.207657e+07,1.395739e+06,6.154515,1.899957e+07,0.659288
3,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,0.606044,8.779783,1002.667748,709.838141,...,21.854187,59.586205,1.610569,386.232189,8.006233,2.240750e+07,1.409503e+06,6.196868,1.913991e+07,0.663482
4,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,0.633401,18.249461,1002.210356,1070.570995,...,21.489489,62.656563,1.453108,445.515113,7.980382,2.268410e+07,1.390317e+06,6.221036,1.922183e+07,0.667129
5,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,0.587619,24.309801,1002.245680,1279.020349,...,21.357787,67.391458,1.313538,510.644488,7.951466,2.296156e+07,1.400359e+06,6.253673,1.932330e+07,0.670241
6,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,0.566759,23.280755,1002.158424,1397.650069,...,21.276233,69.383573,1.242112,564.654167,7.981121,2.318896e+07,1.418401e+06,6.277339,1.939333e+07,0.672829
7,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,0.593537,19.186561,1002.280740,1437.549659,...,21.241476,70.066176,1.226949,595.360558,7.946167,2.385426e+07,1.441334e+06,6.336644,1.960635e+07,0.674904
8,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,0.600821,18.109353,1002.394173,1442.944616,...,21.011926,69.081412,1.247464,612.063590,7.934312,2.447423e+07,1.448568e+06,6.385667,1.978514e+07,0.676479
9,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,0.604731,18.253900,1002.463449,1405.988954,...,20.652449,67.999911,1.231408,616.463496,9.098665,2.502337e+07,1.450923e+06,6.419744,1.991803e+07,0.677105


In [8]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0     276.711000
1     272.764000
2     333.178000
3     386.585000
4     449.076000
5     536.766000
6     639.535000
7     625.022000
8     687.536000
9     778.779000
10    755.713857
11    794.945486
12    842.687657
13    831.356000
14    626.223000
15    702.437000
16    788.164000
17    826.368000
18    804.417000
19           NaN
Name: Piauí - Consumo de Cimento (t), dtype: float64

In [9]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),...,Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Piauí - Produção de Cimento (t),Piauí - Desemprego,Piauí - PIB - Estadual,Piauí - PIB - Construção Civil,Piauí - PIB - Per Capita,Piauí - PIB - Preços de Mercado,Piauí - IDH
0,2.057317,3.335319,2.503268,1.984588,-1.915550,-2.217697,2.670619,-1.849658,-3.137972,-2.254170,...,3.017572,1.986656,-1.199445,-0.870412,-0.759482,-1.609767,-2.279691,-1.909036,-1.868082,-2.083911
1,1.041262,1.436308,0.974434,0.428265,-1.603622,-1.796936,1.398420,-2.038088,-1.609936,-2.086664,...,2.103395,2.294199,-0.600463,-0.618988,-0.812606,-1.411497,-1.527027,-1.587774,-1.553382,-1.636049
2,-0.382310,0.608392,1.584406,-0.633957,-1.305339,-1.387550,0.057571,-1.957276,0.587504,-1.522219,...,0.208251,-1.104584,2.473650,-0.574002,-0.827643,-1.220572,-0.959475,-1.271914,-1.253703,-1.236089
3,-0.655123,0.362019,0.700171,0.047872,-1.020736,-0.988421,0.196419,-0.954603,0.375564,-0.934055,...,0.229351,-1.914907,2.386233,-0.366222,-0.894281,-1.040671,-0.477405,-0.982926,-0.982635,-0.883077
4,-0.562655,0.214378,-0.043890,-0.213672,-0.749851,-0.597180,0.430422,0.236351,-0.331465,-0.044170,...,0.017672,-1.162677,1.193084,0.067305,-0.913012,-0.890309,-1.149354,-0.818020,-0.824410,-0.576060
5,-0.668618,-0.171237,0.095846,-0.882129,-0.492719,-0.230290,0.038818,0.998530,-0.276862,0.470050,...,-0.058770,-0.002639,0.135515,0.543586,-0.933964,-0.739477,-0.797670,-0.595326,-0.628420,-0.314085
6,-0.307381,-0.482640,-0.506598,-0.409438,-0.249377,0.100246,-0.139605,0.869112,-0.411742,0.762696,...,-0.106106,0.485424,-0.405712,0.938551,-0.912477,-0.615859,-0.165758,-0.433844,-0.493156,-0.096198
7,-0.103220,-0.675189,-0.543480,-0.165137,0.025012,0.397049,0.089437,0.354205,-0.222667,0.861124,...,-0.126280,0.652660,-0.520607,1.163102,-0.937804,-0.254194,0.637432,-0.029186,-0.081704,0.078553
8,-0.057995,-0.608258,-0.108258,-0.384906,0.201698,0.638927,0.151742,0.218730,-0.047324,0.874432,...,-0.259515,0.411395,-0.365158,1.285248,-0.946394,0.082831,0.890796,0.305317,0.263623,0.211121
9,0.191223,-0.550547,-0.852408,-0.436543,0.383423,0.817429,0.185184,0.236909,0.059761,0.783267,...,-0.468164,0.146430,-0.486820,1.317424,-0.102709,0.381351,0.973286,0.537836,0.520297,0.263836


In [10]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0     276.711000
1     272.764000
2     333.178000
3     386.585000
4     449.076000
5     536.766000
6     639.535000
7     625.022000
8     687.536000
9     778.779000
10    755.713857
11    794.945486
12    842.687657
13    831.356000
14    626.223000
15    702.437000
16    788.164000
Name: Piauí - Consumo de Cimento (t), dtype: float64

In [11]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),...,Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Piauí - Produção de Cimento (t),Piauí - Desemprego,Piauí - PIB - Estadual,Piauí - PIB - Construção Civil,Piauí - PIB - Per Capita,Piauí - PIB - Preços de Mercado,Piauí - IDH
17,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,0.828411,0.526443,0.377992,...,-0.496396,-0.170518,-0.679571,-1.746865,1.11282,0.859906,0.717584,0.670939,0.684873,1.034736


In [12]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:split_index + 1]
test_target

17    826.368
Name: Piauí - Consumo de Cimento (t), dtype: float64

In [13]:
train_input.iloc[-(len(train_input)//5):]

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),...,Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Piauí - Produção de Cimento (t),Piauí - Desemprego,Piauí - PIB - Estadual,Piauí - PIB - Construção Civil,Piauí - PIB - Per Capita,Piauí - PIB - Preços de Mercado,Piauí - IDH
14,-1.195037,-0.553813,-0.504695,-0.765492,1.098789,0.71401,-1.705322,0.992899,0.765459,0.310035,...,-0.589053,-0.210098,-0.509698,-1.185030,1.400373,1.231996,0.363973,1.101013,1.123692,1.337504
15,-1.670834,-0.541784,-1.357653,0.164210,1.226007,0.54710,-1.702696,0.795306,0.863660,0.337540,...,-0.505206,-0.118468,-0.569020,-1.621324,1.302001,1.223953,0.423683,1.075324,1.095446,1.305811
16,-0.776357,-0.721152,-1.478508,3.015562,1.346999,0.33274,0.058929,0.478831,0.725289,0.459293,...,-0.583912,-0.367932,-0.440606,-1.859259,1.286548,1.108414,0.590900,0.919316,0.939188,1.232971


In [14]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    validation_size = len(train_input)//5
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=10000,
                      validation_data=(train_input.iloc[-validation_size:], 
                                       train_target.iloc[-validation_size:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [15]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [17]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[3819526993, 4219027135, 3376158397, 3325009816, 2503667466, 2622273065, 109221315, 2211009700, 324567649, 1987932507, 814554180, 2319690855, 2588466015, 1093016355, 493586184, 2256538130, 3446867768, 3628929525, 4284337465, 3976511535, 3198276657, 3548295880, 2890721786, 807692438, 1049112881, 1222692420, 130934093, 1031541944, 477899635, 2642135021, 2086629271, 103288431, 3107896754, 2311542733, 48308118, 4108106472, 1831273660, 2345371645, 1923779165, 1991047941, 2819218993, 2399974111, 3253981947, 721999073, 2387234971, 3011307452, 2615468320, 677992432, 1641675254, 794167843, 4273976363, 2075511847, 483002912, 3329036208, 3745358468, 2741697180, 502218154, 2856011778, 3503466164, 842912952, 2887651870, 887177305, 3313070155, 874441430, 3953250672, 2010242235, 2843632229, 1378766290, 2730811678, 3566019255, 2739740624, 2260483631, 2090554743, 362330534, 626213829, 2810103766, 4181654917, 846738639, 604405884, 3568695535, 2241731775, 1172835209, 1396352538, 3960486239, 2587512189, 2

val_loss: 4385.48095703125


Step: 85 ___________________________________________
val_loss: 4450.20263671875


Step: 86 ___________________________________________
val_loss: 4869.9541015625


Step: 87 ___________________________________________
val_loss: 6663.61181640625


Step: 88 ___________________________________________
val_loss: 4980.01611328125


Step: 89 ___________________________________________
val_loss: 4402.47021484375


Step: 90 ___________________________________________
val_loss: 4817.7958984375


Step: 91 ___________________________________________
val_loss: 4387.52685546875


Step: 92 ___________________________________________
val_loss: 5747.923828125


Step: 93 ___________________________________________
val_loss: 4541.71875


Step: 94 ___________________________________________
val_loss: 3365.029541015625


Step: 95 ___________________________________________
val_loss: 5442.20947265625


Step: 96 ___________________________________________
val_loss: 4554.8657226562

In [18]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 0s 470ms/step - loss: 405485.6875 - val_loss: 483249.8750
Epoch 2/10000
1/1 [==============================] - 0s 15ms/step - loss: 391468.3125 - val_loss: 117240.9375
Epoch 3/10000
1/1 [==============================] - 0s 15ms/step - loss: 170214.5938 - val_loss: 684094.9375
Epoch 4/10000
1/1 [==============================] - 0s 14ms/step - loss: 354656.8750 - val_loss: 48980.0312
Epoch 5/10000
1/1 [==============================] - 0s 14ms/step - loss: 262600.0938 - val_loss: 149485.3594
Epoch 6/10000
1/1 [==============================] - 0s 15ms/step - loss: 227646.9844 - val_loss: 232294.1719
Epoch 7/10000
1/1 [==============================] - 0s 14ms/step - loss: 230203.0000 - val_loss: 115478.4609
Epoch 8/10000
1/1 [==============================] - 0s 14ms/step - loss: 239762.0469 - val_loss: 135138.1094
Epoch 9/10000
1/1 [==============================] - 0s 14ms/step - loss: 240811.0000 - val_loss: 115714.7891
Epoch 10/1

1/1 [==============================] - 0s 14ms/step - loss: 66620.2031 - val_loss: 30900.2812
Epoch 76/10000
1/1 [==============================] - 0s 14ms/step - loss: 64481.5586 - val_loss: 85270.1484
Epoch 77/10000
1/1 [==============================] - 0s 14ms/step - loss: 69363.7109 - val_loss: 38224.9180
Epoch 78/10000
1/1 [==============================] - 0s 14ms/step - loss: 59391.3672 - val_loss: 23603.5078
Epoch 79/10000
1/1 [==============================] - 0s 14ms/step - loss: 63770.4180 - val_loss: 34489.0117
Epoch 80/10000
1/1 [==============================] - 0s 14ms/step - loss: 56126.3320 - val_loss: 39760.3867
Epoch 81/10000
1/1 [==============================] - 0s 14ms/step - loss: 57049.7891 - val_loss: 75729.3516
Epoch 82/10000
1/1 [==============================] - 0s 14ms/step - loss: 65635.7812 - val_loss: 51062.9883
Epoch 83/10000
1/1 [==============================] - 0s 14ms/step - loss: 47790.9688 - val_loss: 37110.5195
Epoch 84/10000
1/1 [==============

Epoch 150/10000
1/1 [==============================] - 0s 14ms/step - loss: 26609.1562 - val_loss: 13554.8086
Epoch 151/10000
1/1 [==============================] - 0s 14ms/step - loss: 25662.1523 - val_loss: 12121.7930
Epoch 152/10000
1/1 [==============================] - 0s 15ms/step - loss: 26610.4121 - val_loss: 12865.8896
Epoch 153/10000
1/1 [==============================] - 0s 15ms/step - loss: 23680.9219 - val_loss: 10189.5850
Epoch 154/10000
1/1 [==============================] - 0s 14ms/step - loss: 24272.4844 - val_loss: 11483.9014
Epoch 155/10000
1/1 [==============================] - 0s 14ms/step - loss: 24946.7266 - val_loss: 9066.6670
Epoch 156/10000
1/1 [==============================] - 0s 14ms/step - loss: 24487.2695 - val_loss: 11525.4775
Epoch 157/10000
1/1 [==============================] - 0s 14ms/step - loss: 24698.6211 - val_loss: 11310.0654
Epoch 158/10000
1/1 [==============================] - 0s 14ms/step - loss: 23502.4395 - val_loss: 10183.4932
Epoch 159/1

Epoch 225/10000
1/1 [==============================] - 0s 14ms/step - loss: 13383.2832 - val_loss: 6040.5835
Epoch 226/10000
1/1 [==============================] - 0s 15ms/step - loss: 13230.8809 - val_loss: 5943.0884
Epoch 227/10000
1/1 [==============================] - 0s 15ms/step - loss: 13135.4482 - val_loss: 5529.3423
Epoch 228/10000
1/1 [==============================] - 0s 14ms/step - loss: 12734.9980 - val_loss: 5715.5405
Epoch 229/10000
1/1 [==============================] - 0s 14ms/step - loss: 12640.0947 - val_loss: 5646.9512
Epoch 230/10000
1/1 [==============================] - 0s 14ms/step - loss: 12493.2656 - val_loss: 5597.7993
Epoch 231/10000
1/1 [==============================] - 0s 14ms/step - loss: 13297.2891 - val_loss: 5822.2896
Epoch 232/10000
1/1 [==============================] - 0s 14ms/step - loss: 12232.2734 - val_loss: 5452.5239
Epoch 233/10000
1/1 [==============================] - 0s 14ms/step - loss: 12086.8838 - val_loss: 5491.9683
Epoch 234/10000
1/1

1/1 [==============================] - 0s 14ms/step - loss: 7205.5459 - val_loss: 4382.9380
Epoch 301/10000
1/1 [==============================] - 0s 14ms/step - loss: 7245.7051 - val_loss: 4388.3696
Epoch 302/10000
1/1 [==============================] - 0s 14ms/step - loss: 7106.7314 - val_loss: 4379.7744
Epoch 303/10000
1/1 [==============================] - 0s 14ms/step - loss: 7135.0503 - val_loss: 4382.4014
Epoch 304/10000
1/1 [==============================] - 0s 14ms/step - loss: 7084.1235 - val_loss: 4379.4824
Epoch 305/10000
1/1 [==============================] - 0s 14ms/step - loss: 7048.7534 - val_loss: 4378.6440
Epoch 306/10000
1/1 [==============================] - 0s 14ms/step - loss: 7051.8628 - val_loss: 4379.4248
Epoch 307/10000
1/1 [==============================] - 0s 14ms/step - loss: 6860.0312 - val_loss: 4377.1421
Epoch 308/10000
1/1 [==============================] - 0s 14ms/step - loss: 6876.6289 - val_loss: 4376.5098
Epoch 309/10000
1/1 [=======================

...

Epoch 908/10000
1/1 [==============================] - 0s 15ms/step - loss: 4710.6006 - val_loss: 8030.1953
Epoch 909/10000
1/1 [==============================] - 0s 14ms/step - loss: 5832.7886 - val_loss: 4458.9390
Epoch 910/10000
1/1 [==============================] - 0s 14ms/step - loss: 5607.8105 - val_loss: 5317.2988
Epoch 911/10000
1/1 [==============================] - 0s 14ms/step - loss: 5071.2393 - val_loss: 5211.4912
Epoch 912/10000
1/1 [==============================] - 0s 14ms/step - loss: 5064.2617 - val_loss: 5246.0708
Epoch 913/10000
1/1 [==============================] - 0s 15ms/step - loss: 5320.2466 - val_loss: 5023.4302
Epoch 914/10000
1/1 [==============================] - 0s 14ms/step - loss: 5304.4458 - val_loss: 5086.1211
Epoch 915/10000
1/1 [==============================] - 0s 14ms/step - loss: 5058.4072 - val_loss: 5300.1113
Epoch 916/10000
1/1 [==============================] - 0s 14ms/step - loss: 5308.3071 - val_loss: 5026.2974
Epoch 917/10000
1/1 [=======

Epoch 984/10000
1/1 [==============================] - 0s 14ms/step - loss: 5788.2329 - val_loss: 5049.9043
Epoch 985/10000
1/1 [==============================] - 0s 14ms/step - loss: 5780.7944 - val_loss: 5052.9663
Epoch 986/10000
1/1 [==============================] - 0s 14ms/step - loss: 5773.4648 - val_loss: 5055.9902
Epoch 987/10000
1/1 [==============================] - 0s 14ms/step - loss: 5766.2456 - val_loss: 5058.9863
Epoch 988/10000
1/1 [==============================] - 0s 14ms/step - loss: 5759.1313 - val_loss: 5061.9473
Epoch 989/10000
1/1 [==============================] - 0s 14ms/step - loss: 5752.1226 - val_loss: 5064.8662
Epoch 990/10000
1/1 [==============================] - 0s 14ms/step - loss: 5745.2173 - val_loss: 5067.7563
Epoch 991/10000
1/1 [==============================] - 0s 14ms/step - loss: 5738.4121 - val_loss: 5070.6138
Epoch 992/10000
1/1 [==============================] - 0s 14ms/step - loss: 5731.7080 - val_loss: 5073.4351
Epoch 993/10000
1/1 [=======

1/1 [==============================] - 0s 14ms/step - loss: 5366.0708 - val_loss: 5206.9912
Epoch 1060/10000
1/1 [==============================] - 0s 14ms/step - loss: 5362.3906 - val_loss: 5233.2056
Epoch 1061/10000
1/1 [==============================] - 0s 14ms/step - loss: 5360.0898 - val_loss: 5227.1680
Epoch 1062/10000
1/1 [==============================] - 0s 13ms/step - loss: 5357.9224 - val_loss: 5229.9199
Epoch 1063/10000
1/1 [==============================] - 0s 14ms/step - loss: 5218.1123 - val_loss: 4563.7310
Epoch 1064/10000
1/1 [==============================] - 0s 14ms/step - loss: 6010.5513 - val_loss: 5136.2402
Epoch 1065/10000
1/1 [==============================] - 0s 14ms/step - loss: 5852.7993 - val_loss: 5073.9800
Epoch 1066/10000
1/1 [==============================] - 0s 14ms/step - loss: 5911.7578 - val_loss: 5047.2007
Epoch 1067/10000
1/1 [==============================] - 0s 14ms/step - loss: 5446.3789 - val_loss: 5219.1060
Epoch 1068/10000
1/1 [==============

1/1 [==============================] - 0s 14ms/step - loss: 5319.0156 - val_loss: 5231.1904
Epoch 1135/10000
1/1 [==============================] - 0s 14ms/step - loss: 5317.8457 - val_loss: 5231.5513
Epoch 1136/10000
1/1 [==============================] - 0s 14ms/step - loss: 5316.6895 - val_loss: 5231.8940
Epoch 1137/10000
1/1 [==============================] - 0s 14ms/step - loss: 5315.5459 - val_loss: 5232.2368
Epoch 1138/10000
1/1 [==============================] - 0s 14ms/step - loss: 5314.4155 - val_loss: 5232.5659
Epoch 1139/10000
1/1 [==============================] - 0s 14ms/step - loss: 5313.2988 - val_loss: 5232.8911
Epoch 1140/10000
1/1 [==============================] - 0s 14ms/step - loss: 5312.1934 - val_loss: 5233.2056
Epoch 1141/10000
1/1 [==============================] - 0s 14ms/step - loss: 5311.1025 - val_loss: 5233.5127
Epoch 1142/10000
1/1 [==============================] - 0s 14ms/step - loss: 5310.0229 - val_loss: 5233.8130
Epoch 1143/10000
1/1 [==============

1/1 [==============================] - 0s 14ms/step - loss: 5227.2095 - val_loss: 5344.2964
Epoch 1210/10000
1/1 [==============================] - 0s 14ms/step - loss: 5285.6562 - val_loss: 5151.3208
Epoch 1211/10000
1/1 [==============================] - 0s 14ms/step - loss: 5237.2456 - val_loss: 5394.0420
Epoch 1212/10000
1/1 [==============================] - 0s 14ms/step - loss: 5227.3462 - val_loss: 5322.2422
Epoch 1213/10000
1/1 [==============================] - 0s 14ms/step - loss: 5226.2446 - val_loss: 5339.0874
Epoch 1214/10000
1/1 [==============================] - 0s 14ms/step - loss: 5285.6284 - val_loss: 5147.0063
Epoch 1215/10000
1/1 [==============================] - 0s 14ms/step - loss: 5236.2695 - val_loss: 5388.6855
Epoch 1216/10000
1/1 [==============================] - 0s 14ms/step - loss: 5226.4253 - val_loss: 5317.2578
Epoch 1217/10000
1/1 [==============================] - 0s 14ms/step - loss: 5283.8940 - val_loss: 5149.0513
Epoch 1218/10000
1/1 [==============

In [19]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 48ms/step
[826.368](test_target) - [[751.396]](prediction) = 74.97200390625005


In [20]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [21]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 9, winner_seed)
predictions

train_input:


,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),...,Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Piauí - Produção de Cimento (t),Piauí - Desemprego,Piauí - PIB - Estadual,Piauí - PIB - Construção Civil,Piauí - PIB - Per Capita,Piauí - PIB - Preços de Mercado,Piauí - IDH
0,2.184461,2.272299,1.914650,2.360960,-1.590751,-1.593283,2.299346,-1.071845,-2.225635,-1.394036,...,2.090883,1.298940,-1.234230,-1.260938,1.865075,-1.539675,-1.748395,-1.648252,-1.663210,-1.736497
1,1.077411,0.707092,0.380286,0.491065,-1.095308,-1.106645,0.891536,-1.226375,-0.870458,-1.253431,...,1.279900,1.516950,-0.781301,-0.910929,0.999186,-1.060422,-0.834425,-1.094089,-1.086381,-1.109588
2,-0.473651,0.024704,0.992464,-0.785177,-0.621537,-0.633164,-0.592240,-1.160102,1.078396,-0.779633,...,-0.401319,-0.892371,1.543230,-0.848304,0.754096,-0.598923,-0.145240,-0.549245,-0.537085,-0.549730
3,-0.770896,-0.178361,0.105030,0.034030,-0.169496,-0.171546,-0.438592,-0.337816,0.890432,-0.285926,...,-0.382601,-1.466791,1.477128,-0.559051,-0.332049,-0.164072,0.440144,-0.050753,-0.040230,-0.055589
4,-0.670147,-0.300051,-0.641723,-0.280211,0.260758,0.280950,-0.179645,0.638877,0.263386,0.461048,...,-0.570385,-0.933552,0.574913,0.044464,-0.637362,0.199378,-0.375812,0.233702,0.249788,0.374169
5,-0.785600,-0.617883,-0.501481,-1.083350,0.669167,0.705281,-0.612992,1.263934,0.311812,0.892687,...,-0.638199,-0.111227,-0.224782,0.707498,-0.978865,0.563966,0.051243,0.617841,0.609028,0.740877
6,-0.392011,-0.874548,-1.106104,-0.515420,1.055674,1.087568,-0.810435,1.157799,0.192190,1.138335,...,-0.680191,0.234750,-0.634039,1.257330,-0.628630,0.862772,0.818581,0.896389,0.856960,1.045872
7,-0.169566,-1.033252,-1.143120,-0.221898,1.491492,1.430840,-0.556978,0.735528,0.359876,1.220955,...,-0.698088,0.353300,-0.720918,1.569929,-1.041451,1.736977,1.793905,1.594406,1.611130,1.290485


train_target:


,Piauí - Consumo de Cimento (t)
0,276.711
1,272.764
2,333.178
3,386.585
4,449.076
5,536.766
6,639.535
7,625.022


test_input:


...

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),...,Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Piauí - Produção de Cimento (t),Piauí - Desemprego,Piauí - PIB - Estadual,Piauí - PIB - Construção Civil,Piauí - PIB - Per Capita,Piauí - PIB - Preços de Mercado,Piauí - IDH
17,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,0.828411,0.526443,0.377992,...,-0.496396,-0.170518,-0.679571,-1.746865,1.11282,0.859906,0.717584,0.670939,0.684873,1.034736


test_target:


,Piauí - Consumo de Cimento (t)
17,826.368


1/1 [==============================] - 0s 35ms/step
Error: 74.97200390625005




[566.87109375,
 622.9237060546875,
 662.5416259765625,
 685.3017578125,
 738.3689575195312,
 708.0218505859375,
 746.6768798828125,
 745.55712890625,
 702.4452514648438,
 751.39599609375]

In [22]:
display(targets)
display(predictions)

[687.536,
 778.779,
 755.7138571428571,
 794.9454857142857,
 842.6876571428572,
 831.356,
 626.223,
 702.437,
 788.164,
 826.368]

[566.87109375,
 622.9237060546875,
 662.5416259765625,
 685.3017578125,
 738.3689575195312,
 708.0218505859375,
 746.6768798828125,
 745.55712890625,
 702.4452514648438,
 751.39599609375]

In [23]:
mae = mean_absolute_error(predictions, targets)
mae

103.12537695312501

In [24]:
porcentage = mae/np.mean(targets)
porcentage

0.13508323317425772